# Notebook 01: Environment Setup

This notebook sets up the complete environment for LLM finetuning in Google Colab.

**What this notebook does:**
1. Check GPU availability and specs
2. Mount Google Drive for checkpoint persistence
3. Clone GitHub repository
4. Install all required dependencies
5. Configure experiment tracking (W&B, TensorBoard)
6. Verify installation
7. Test basic functionality

**Estimated time:** 5-10 minutes

## 1. Check GPU Availability

In [ ]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except:
    IN_COLAB = False
    print("✗ Not running in Colab")

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"Total GPU Memory: {total_memory:.2f} GB")
    
    # Determine GPU type
    gpu_name = torch.cuda.get_device_name(0)
    if 'T4' in gpu_name:
        print("\n📌 GPU Type: Tesla T4 (Good for most models with quantization)")
    elif 'V100' in gpu_name:
        print("\n📌 GPU Type: Tesla V100 (Excellent for all models)")
    elif 'A100' in gpu_name:
        print("\n📌 GPU Type: A100 (Best performance, can use bf16)")
else:
    print("\n⚠️  WARNING: No GPU detected!")
    print("Go to: Runtime → Change runtime type → Hardware accelerator → GPU")

## 2. Mount Google Drive

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✓ Google Drive mounted")
    
    # Create checkpoint directory
    import os
    checkpoint_dir = "/content/drive/MyDrive/llm_finetuning_checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)
    print(f"✓ Checkpoint directory: {checkpoint_dir}")
else:
    print("Skipping Drive mount (not in Colab)")

## 3. Clone GitHub Repository

In [ ]:
# Replace with your GitHub repository URL
REPO_URL = "https://github.com/YOUR_USERNAME/llm-finetuning-production.git"

# Clone repo if not already present
import os

if not os.path.exists("llm-finetuning-production"):
    !git clone {REPO_URL}
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

# Change to project directory
%cd llm-finetuning-production

## 4. Install Dependencies

In [ ]:
# Install all requirements
!pip install -q -r requirements.txt

print("✓ Dependencies installed")

## 5. Verify Installation

In [ ]:
# Verify key libraries
import transformers
import datasets
import peft
import trl
import accelerate
import evaluate

print("Library Versions:")
print(f"  transformers: {transformers.__version__}")
print(f"  datasets: {datasets.__version__}")
print(f"  peft: {peft.__version__}")
print(f"  trl: {trl.__version__}")
print(f"  accelerate: {accelerate.__version__}")

# Check bitsandbytes (for quantization)
try:
    import bitsandbytes as bnb
    print(f"  bitsandbytes: {bnb.__version__}")
except:
    print("  bitsandbytes: ✗ Not available (quantization will fail)")

print("\n✓ All core libraries installed successfully!")

## 6. Configure Experiment Tracking

In [ ]:
# Configure Weights & Biases
import wandb

# Option 1: Login with API key
# wandb.login(key="YOUR_WANDB_API_KEY")

# Option 2: Login interactively
# wandb.login()

print("📊 W&B setup: Run wandb.login() when ready to track experiments")

In [ ]:
# Configure HuggingFace (for downloading gated models)
from huggingface_hub import login

# Option 1: Login with token
# login(token="YOUR_HF_TOKEN")

# Option 2: Login interactively
# login()

print("🤗 HuggingFace setup: Run login() for gated models (Llama, Mistral, etc.)")

## 7. Test Basic Functionality

In [ ]:
# Test our custom modules
import sys
sys.path.append('/content/llm-finetuning-production')

from src.utils.memory import print_gpu_utilization, get_gpu_memory_info

print("\n=== GPU Memory Status ===\n")
print_gpu_utilization()

mem_info = get_gpu_memory_info()
print(f"\nAvailable Memory: {mem_info['available']:.2f} GB")

In [ ]:
# Test loading a small model
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Testing model loading with GPT-2 (small)...\n")

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move to GPU
if torch.cuda.is_available():
    model = model.to("cuda")

print(f"✓ Model loaded: {model_name}")
print(f"✓ Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Test generation
text = "Hello, I am"
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=20)
generated = tokenizer.decode(outputs[0])

print(f"\nTest generation:")
print(f"  Input: {text}")
print(f"  Output: {generated}")

# Clean up
del model
del tokenizer
torch.cuda.empty_cache()

print("\n✓ Model loading and generation test successful!")

## 8. Environment Summary

In [ ]:
print("=" * 60)
print("ENVIRONMENT SETUP COMPLETE")
print("=" * 60)

print("\n✓ GPU: Available and tested")
print("✓ Dependencies: Installed")
print("✓ Custom modules: Working")
print("✓ Model loading: Tested")

print("\n📝 Next Steps:")
print("  1. (Optional) Configure W&B: wandb.login()")
print("  2. (Optional) Configure HF: huggingface_hub.login()")
print("  3. Move to Notebook 02: Data Exploration")

print("\n" + "=" * 60)